# **Trabajo Practico Numero 1**

**Antes que nada**:</br> $para$ $los$ $ejercicios$ $seran$ $necesarias$ $las$ $librerias$ **numpy**, **scipy** $y$ **matplotlib**, </br>$por$ $lo$ $tanto$, $las$ $importamos$

In [8]:
import numpy as np
import matplotlib as plt
import scipy as sp

## Ejercicio 1

Dada una matriz A $\in \mathbb R ^{n \times n}$, devuelve las dos matrices L y U de la descomposición A = LU.

In [9]:
def descompLU(A):
#Si hay algún 0 en la diagonal de A, imprime un mensaje de error y devuelve las matrices In y A. 
for p in range (0,len(A)): 
     if A[p][p]==0: 
         I = np.eye(len(A)) 
         return ('error, In= ', I , 'A= ', A) 
#Creamos 2 matrices de 0s L y U del mismo tamaño que A. 
L=np.zeros((len(A),len(A[0]))) 
U=np.zeros((len(A),len(A[0]))) 

for i in range(0,len(A)): 
#La diagional de L es toda de 1s. 
    L[i][i]=1 
    j=0 
    m=i 
#Genera recursivamente los valores de L por debajo de la diagonal en la fila i. 
    while j < i: 
         L[i][j] = A[i][j]/U[j][j] 
         k=0 
         while k < j: 
             L[i][j]=L[i][j]-(L[i][k]*U[k][j]/U[j][j]) 
             k=k+1 
         j=j+1 
#Genera recursivamente los valores de U que no son 0 en la fila i. 
     while m<len(A):
         U[i][m] = A[i][m] 
         n=0 
         while n < i: 
             U[i][m]=U[i][m]-L[i][n]*U[n][m] 
             n=n+1 
         m=m+1 
return(L,U) 


Chequeamos la función con 2 ejemplos.

In [10]:
A = np.array([[4, 3, 2, 1],
              [2, 9, 7, 5],
              [4, 7, 7, 8],
              [6, 9, 8, 9]])

B= np.array([[0,1],[1,2]]),

L, U = descompLU(A)
print("Matriz L de A:")
print(L)
print("\nMatriz U de A:")
print(U)

print("\nMatriz LU de A:\n", L@U) 
print("\nMatriz LU de B:\n", descompLU(B))

Matriz L de A:
[[1.        , 0.        , 0.        , 0.        ] 
 [0.5       , 1.        , 0.        , 0.        ] 
 [1.        , 0.53333333, 1.        , 0.        ] 
 [1.5       , 0.6       , 0.77777778, 1.        ]] 

Matriz U de A:
[[4.        , 3.        , 2.        , 1.        ] 
 [0.        , 7.5       , 6.        , 4.5       ] 
 [0.        , 0.        , 1.8       , 4.6       ] 
 [0.        , 0.        , 0.        , 1.22222222]] 

Matriz LU de A:
[[4., 3., 2., 1.] 
 [2., 9., 7., 5.] 
 [4., 7., 7., 8.] 
 [6., 9., 8., 9.]] 

Matriz LU de B:
('error, In= ',
 array([[1., 0.],
       [0., 1.]]),
 'A= ',
 array([[0, 1],
        [1, 2]]))


## Ejercicio 3

In [11]:
def resolverLU(A, b):
    '''
    Parametros:
        *matriz A n*n
        *vector b de n elementos
    Retorna:
        *x vector solucion de la ecuación Ax = b
    '''

    #Obtenemos L y U haciendo uso del ejercicio 1
    L, U = descompLU(A)

    #Resuelvo la ecuacion Ly = b especificada en la consigna
    y = sp.linalg.solve_triangular(L, b)

    #Con (y) puedo obtener la solucion a la ecuación Ux = y
    x = sp.linalg.solve_triangular(U, y)

    return x


In [12]:
# Ejemplo de uso
A = np.array([[4, 3, 2, 1],
              [2, 9, 7, 5],
              [4, 7, 7, 8],
              [6, 9, 8, 9]])

b = np.array([4,4,7,8])

x = resolverLU(A, b)
print(x)

[ -0.625 -10.5    23.     -8.   ]


## Ejercicio 5

In [13]:
def inversa(A):
    '''
    Parametro: matriz A n*n
    Retorna: matriz inversa de A
    '''

    #Guardamos el numero de filas de A en una variable n
    #Obs: n tambien es el numero de columnas, pues A es cuadrada
    n = A.shape[0]

    #Paso 1: Descomponemos A en LU
    L, U = descompLU(A)

    #Uso la matriz identidad_n (I) para tener los vectores e_i
    I = np.eye(n)

    #Armo una matriz llena de 0's, para luego agregarle las columnas correspondientes
    B = np.zeros_like(A)

    # 1 Uso un ciclo for para obtener las soluciones al sistema Ax = e_i con 1 <= i <=n
    # 2 Cada e_i es extraido de las filas de la matriz I
    # 3 Resuelvo el sistema Ly = e_i
    # 4 Resuelvo el sistema Ux = y
    # 5 Asigno la columna (x) al lugar donde corresponde en B
    for i in range(n): # --> 1
        e_i = I[i,:] # --> 2
        y = sp.linalg.solve_triangular(L, e_i) # --> 3
        x = sp.linalg.solve_triangular(U, y) # --> 4
        B[:,i] = x # --> 5

    #terminadas las iteraciones tengo que B es la matriz inversa de A
    return B
        


In [14]:
# Ejemplo de uso
A = np.array([[4, 3, 2, 1],
              [2, 9, 7, 5],
              [4, 7, 7, 8],
              [6, 9, 8, 9]])

B = inversa(A)
print(B)

[[ 0  0  0  0]
 [ 0  0  0 -1]
 [ 0  0  1  2]
 [ 0  0  0 -1]]
